# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [6]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [9]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Sit', 'Crouch', 'Stand', 'Left', 'Belly', 'Knee', 'Right', 'StandInit', 'Back', 'HeadBack', 'Frog']


In [15]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    
    # YOUR CODE HERE
    ROBOT_POSE_DATA_DIR = 'robot_pose_data'
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename, 'rb'))
    target = [i] * len(data)
    return data, target

In [16]:
# load all the data
all_data = []
all_target = []

# YOUR CODE HERE
for i in range(len(classes)):
    d, t = load_pose_data(i)
    all_data.extend(d)
    all_target.extend(t)

print('total number of data', len(all_data))

total number of data 222


In [17]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [18]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [19]:
# YOUR CODE HERE
all_data_arr = np.array(all_data)
all_target_arr = np.array(all_target)
clf.fit(all_data_arr[training_data], all_target_arr[training_data])

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

### predicting

In [23]:
clf.predict(np.array(all_data[-1]).reshape(1, -1)), all_target[-1]

(array([10]), 10)

In [25]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [26]:
expected = []
predicted = []

# YOUR CODE HERE
expected = all_target_arr[training_data]
predicted = clf.predict(all_data_arr[training_data]) 

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       0.96      1.00      0.98        24
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00        16
           4       1.00      1.00      1.00        12
           5       1.00      1.00      1.00         8
           6       1.00      0.86      0.92         7
           7       1.00      1.00      1.00        32
           8       1.00      1.00      1.00        16
           9       1.00      1.00      1.00         6
          10       1.00      1.00      1.00         6

    accuracy                           0.99       155
   macro avg       1.00      0.99      0.99       155
weighted avg       0.99      0.99      0.99       155


Confusion matrix:
[[20  0  0  0  0  0  0  0  0  0  0]
 [ 0 24  0  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0 16  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [27]:
expected = []
predicted = []

# YOUR CODE HERE
predicted = clf.predict(all_data_arr[permutation[n_training_data:]])
expected = all_target_arr[permutation[n_training_data:]]

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         4
           4       1.00      1.00      1.00         7
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         4
           7       1.00      1.00      1.00        20
           8       1.00      1.00      1.00         7
           9       1.00      1.00      1.00         4
          10       1.00      1.00      1.00         4

    accuracy                           1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67


Confusion matrix:
[[ 6  0  0  0  0  0  0  0  0  0  0]
 [ 0  6  0  0  0  0  0  0  0  0  0]
 [ 0  0  3  0  0  0  0  0  0  0  0]
 [ 0  0  0  4  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [52]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [34]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict(np.array(all_data[-1]).reshape(1, -1)), all_target[-1]

(array([10]), 10)